#### <img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 4:  Using Yelp cost estimates for estimating neighborhood affluency

<i>
                
                Submitted by Shannon Bingham and Roy Kim
</i>

 
## Problem Statement

This tool will estimate the affluence of a neighborhood based on the number of $ of businesses and services (according to Yelp) in a given neighborhood. ($, $$, $$$) This tool will expect to get, as an input, a list of zip codes or names of neighborhoods and will estimate the wealth of the locality. While traditional methods typically estimate wealth of a locality based on demographic characteristics (e.g. income or unemployment rate), the novelty of this approach is in its use of big data related to commercial activity and cost of product and services as an indicator for affluency.


## Set up environment.

In [1]:
# Import libraries.
import requests
import time
import pandas as pd

## Read in positioning data.

In [2]:
# Locate the file.
next_post_csv = './data/next_post_20181216-231653.csv'
# next_post_csv = './data/next_post_init.csv'

# Load the data.  
next_post_df = pd.read_csv(next_post_csv, index_col='url')

# Take a look. 
next_post_df.head()

In [3]:
def get_details(the_json, keys):
# Get the details of each reddit post in the json data.  
# Return the details in a list. 

    # Set details (dictionary keys) to select.  
    select_keys = ['title', 'subreddit', 'author', 'domain', 
                   'num_comments', 'permalink']

    # Initialize list.
    posts = []

    # Loop through the posts for selected dictionary key values.
    for i in range(len(the_json['data']['children'])):
        
        posts.extend([{k : the_json['data']['children'][i]['data'][k] 
                       for k in keys}])
    
    # Return details.
    return posts

## Request posts from reddit website using API.

In [12]:
# Initialize lists.
urls = ['https://www.reddit.com/r/news.json']
        'https://www.reddit.com/r/UpliftingNews.json']

all_posts = []

# Set up header for API.
headers = {'User-agent': 'Bingham v1.0'}

# Loop through urls.
for url in urls:

    # Set number of requests to make.
    n_requests = 40

    # Set details (dictionary keys) to select from reddit posts.  
    select_keys = ['title', 'name', 'subreddit', 'author', 'domain', 
                   'num_comments', 'permalink']
    
    # Print progress message.
    print(url)
    print('Request processing starting')
    
    after = 'init'

    # Loop for each request (note that each request returns up to 25 posts).
    for n in range(n_requests):

        # Print progress message.
        print('Making request #', n)

        # Set variable to indicate the id of the next post.
        after = next_post_df.loc[url,'after']
        
        if after == 'init':
            params = {}
        else:
            params = {'after': after}

        # Make request.      
        res = requests.get(url, params=params, headers=headers)

        # Process data.
        if res.status_code == 200:     # successful request
              
            # Get the details from all the returned posts.
            the_json = res.json()
            all_posts.extend(get_details(the_json, select_keys))
            
            # Prepare for next request.
            after = the_json['data']['after']
            next_post_df.loc[url,'after'] = after
            
            print("Last post in request:", after)
        
        else:                          # unsuccessful request
            print('Processing ended unexpectedly.') 
            print('Request.get response is ', res.status_code)
            print('url is ', url)
            break

        # Wait.
        time.sleep(3)
         
    # Print progress message.
    print('Request processing ended')

https://www.reddit.com/r/UpliftingNews.json
Request processing starting
Making request # 0
Last post in request: t3_a70j15
Making request # 1
Last post in request: t3_a65964
Making request # 2
Last post in request: t3_a652sf
Making request # 3
Last post in request: t3_a5b6mf
Making request # 4
Last post in request: t3_a5cb5k
Making request # 5
Last post in request: t3_a5101s
Making request # 6
Last post in request: t3_a4s11e
Making request # 7
Last post in request: t3_a4j9xl
Making request # 8
Last post in request: t3_a49t5y
Making request # 9
Last post in request: t3_a3r903
Making request # 10
Last post in request: t3_a3hrsy
Making request # 11
Last post in request: t3_a2rrnz
Making request # 12
Last post in request: t3_a2j1qe
Making request # 13
Last post in request: t3_a1qa0b
Making request # 14
Last post in request: t3_a17fgx
Making request # 15
Last post in request: t3_a0x903
Making request # 16
Last post in request: t3_a059g2
Making request # 17
Last post in request: t3_9z3qaj
Ma

In [13]:
len(all_posts)


966

In [14]:
# Check for duplicate titles.
len(set([p['title']for p in all_posts]))

482

In [15]:
# Load posts to a dataframe.
all_posts_df = pd.DataFrame(all_posts, columns = select_keys)

# Verify load.
all_posts_df.shape

(966, 7)

In [9]:
# Take a look.
all_posts_df.tail(50)

title       name subreddit  \
902  Stethoscopes loaded with bacteria, including s...  t3_a6i0ro      news   
903  Israel destroys house of Palestinian charged w...  t3_a6gwwq      news   
904  Sandy Hook School Students Sent Home After Rec...  t3_a65xzu      news   
905  Big data hints at how, when and where mental d...  t3_a6exk4      news   
906  Special forces solider charged with murder; "a...  t3_a69ix8      news   
907  Nurse Denied Life Insurance Because She Carrie...  t3_a64ca7      news   
908  Texas judge who approved plea deal for alleged...  t3_a654l5      news   
909  Dept. of Education to cancel $150 million in s...  t3_a64nxv      news   
910  Three Men Indicted In Conspiracy to Kill Whist...  t3_a683s9      news   
911  J&amp;J shares plunge as much as 8.6% after re...  t3_a668f3      news   
912  A Oregon veteran is fighting for his life, wai...  t3_a68yfk      news   
913  Police face first ‘super-complaint’ over immig...  t3_a6hf49      news   
914  Michigan bill to ban growing marijuana at home...  t3_a64v3i      news   
915  CBP: Father of Guatemalan girl who died in cus...  t3_a67brq      news   
916       Hartwell man charged with rape of 5-year-old  t3_a6cf1v      news   
917   Banned in Boston: Plastic bags at grocery stores  t3_a6636a      news   
918  U.S. officials pick Salt Lake City over Denver...  t3_a6ab9y      news   
919  4,000-Year-Old Game Board Carved into the Eart...  t3_a63cmj      news   
920  Court: Florida police can use 'stand your grou...  t3_a6bdfm      news   
921  British man, 38, who has worked his whole life...  t3_a66yhu      news   
922  Marine Corps says $3.6B for storm repairs at C...  t3_a6bbgw      news   
923  Tesla employees launch organizing drive with h...  t3_a667f3      news   
924  Tech Giants Warn US Govt. Against EU's 'Articl...  t3_a6b2e0      news   
925  ICE arrests immigrant husband of disabled Army...  t3_a6119d      news   
926  High-schooler expelled for cursing hires civil...  t3_a69rq4      news   
927  WaPost: Girl dies of dehydration while in Bord...  t3_a5zya6      news   
928  Judge rejects embattled school deputy's claim ...  t3_a5zdjc      news   
929  Facebook discovers bug that may have affected ...  t3_a663tv      news   
930  CBS Paid the Actress Eliza Dushku $9.5 Million...  t3_a622qn      news   
931  Massachusetts cannabis panel OKs 4 more pot sh...  t3_a67c7r      news   
932  Montana man pleads guilty to double murder, pu...  t3_a696vp      news   
933  Major Project To Identify Long Nameless Korean...  t3_a64o0c      news   
934  Woman accused of killing two newborns and disp...  t3_a67qau      news   
935  Emblems of hate seized from group in racist Se...  t3_a677rb      news   
936  Miss USA apologizes for making fun of Miss Vie...  t3_a680cw      news   
937  Nebraska senators want to legalize marijuana v...  t3_a5w4w9      news   
938    Delta Warns Unhappy Employees About Job Actions  t3_a68hv8      news   
939  The year's brightest comet streaks by Earth th...  t3_a66goj      news   
940  Euro falls as 'gilets jaunes' protests hit Fre...  t3_a641mr      news   
941  Judge halts Keystone XL pipeline, citing ‘comp...  t3_a5uz2j      news   
942  Top secret report: North Korea keeps busting s...  t3_a66gje      news   
943  California state assemblyman arrested after da...  t3_a64zua      news   
944  Man in ‘cocaine’ shirt accused of graffitiing ...  t3_a5yn95      news   
945  Marriott Data Breach Is Traced to Chinese Hack...  t3_a67eqs      news   
946  Richmond school shooting: Suspect in attack id...  t3_a67qoc      news   
947  Alleged Russian spy Maria Butina pleads guilty...  t3_a5v8vp      news   
948  The Weekly Standard, a conservative magazine c...  t3_a66r0i      news   
949  Iran says U.N.-led ceasefire in Yemen's Hodeid...  t3_a6992c      news   
950   Debt collector reaches settlement with 41 states  t3_a65hdg      news   
951      Barn fire kills 1,300 pigs in eastern Ontario  t3_a669mv      news   

                   

In [10]:
# Take a look at next post data.
# next_post_df

## Save files.

In [16]:
timestamp = time.strftime("%Y%m%d-%H%M%S")

# Set file locations.
all_posts_csv = (f'./data/all_posts_{timestamp}.csv')

# Save.
next_post_df.to_csv(next_post_csv, encoding='utf-8')
all_posts_df.to_csv(all_posts_csv, encoding='utf-8', index=False)